In [ ]:
# Installation Library

!pip install transformers datasets sentencepiece rouge 

In [ ]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
dataset = load_dataset("billsum")

Using custom data configuration default
Reusing dataset billsum (/root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
test_cases =  dataset['ca_test']

In [ ]:
case1 = test_cases[1]['text']
sum1 =  test_cases[1]['summary']

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "nsi319/legal-pegasus"

tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [ ]:
batch = tokenizer.encode(case1, truncation=True, padding="longest", return_tensors="pt").to(device)

In [ ]:
summary1 = model.generate(batch, max_length=2000, min_length=500, num_beams=9,)
systemsummary = tokenizer.batch_decode(summary1, skip_special_tokens=True)

In [ ]:
standard_summary = sum1
system_summary = systemsummary[0]

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, standard_summary)
pd.DataFrame(score[0]).set_index([['recall','precision','f-measure']])

,rouge-1,rouge-2,rouge-l
recall,0.489796,0.275449,0.479592
precision,0.289157,0.158621,0.283133
f-measure,0.363636,0.201313,0.356061


---------

**Test DataSet**

In [ ]:
CasesText = test_cases['text']
GoldSummary = test_cases['summary']

In [ ]:
len(CasesText), len(GoldSummary)

(1237, 1237)

In [ ]:
# CasesText[:188]
# CasesText[188:300]
# CasesText[300:1000]
# CasesText[1000:]

In [ ]:
SystemSummary = []

for i, case in enumerate(CasesText[1000:]):
    
    batch = tokenizer.encode(case, truncation=True, padding="longest", return_tensors="pt").to(device)
    
    summary = model.generate(batch, min_length=1024, num_beams=9)
    Summary = tokenizer.batch_decode(summary, skip_special_tokens=True)
    
    SystemSummary.append(Summary)
    print(i)

In [ ]:
len(SystemSummary), len(GoldSummary[1000:])

(237, 237)

In [ ]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary[1000:], SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [ ]:
path = "/content/drive/MyDrive/"

Summaries.to_csv(path + "LegalPegsus4.csv", header=True, index=False)

In [ ]:
file1 = pd.read_csv(path + "LegalPegsus1.csv")
file2 = pd.read_csv(path + "LegalPegsus2.csv")
file3 = pd.read_csv(path + "LegalPegsus3.csv")
file4 = pd.read_csv(path + "LegalPegsus4.csv")

In [ ]:
LegalPegsusSummaries = pd.concat([file1, file2, file3, file4])

In [ ]:
LegalPegsusSummaries.sample(5)

,GoldSummary,SystemSummary
456,Existing law establishes the Department of Tec...,The California Open Data Act of 2015 requires ...
570,Existing law with respect to claims against pu...,The California Department of General Services ...
135,The Child Abuse and Neglect Reporting Act requ...,"A county welfare agency, as determined in Sect..."
67,"Existing law, as amended by Proposition 9, the...",The California Board of Parole Hearings is req...
140,The County Sanitation District Act authorizes ...,The Los Angeles County Sanitation District (Sa...


In [ ]:
system_summary = LegalPegsusSummaries['SystemSummary']
standard_summary = LegalPegsusSummaries['GoldSummary']

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, standard_summary, avg=True)

In [ ]:
LegalPegsusRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])*100

In [ ]:
LegalPegsusRouge

,rouge-1,rouge-2,rouge-l
recall,22.614598,9.324869,20.427144
precision,45.216152,21.799144,41.045372
f-measure,28.619040,12.196451,25.887972


In [ ]:
LegalPegsusRouge.to_csv("LegalPegsusRouge.csv", header=True)